# CNN x ECG on PTB-XL Dataset

tbd description

## Import

In [56]:
import ast
import wfdb
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split

# --- Simple NN ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# --- xresnet1d101 ---
from tsai.all import *
from fastai.data.all import DataLoaders
from fastai.learner import Learner
# from fastai.metrics import accuracy_thresh
from fastai.callback.all import *

## Load Data

tbd description

In [73]:
path = 'data/ptbxl/'

### Metadata

tbd description

In [74]:
df_metadata = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')

In [75]:
# add the base path to the filenames
df_metadata['filename_lr'] = df_metadata['filename_lr'].apply(lambda x: path + x)
df_metadata['filename_hr'] = df_metadata['filename_hr'].apply(lambda x: path + x)

In [76]:
# convert scp_codes strings, like "{'NORM': 100}", to dictionaries like {'NORM': 100}
df_metadata['scp_codes'] = df_metadata['scp_codes'].apply(lambda x: ast.literal_eval(x))

In [77]:
# The Y for machine learning
df_metadata

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,data/ptbxl/records100/00000/00001_lr,data/ptbxl/records500/00000/00001_hr
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,data/ptbxl/records100/00000/00002_lr,data/ptbxl/records500/00000/00002_hr
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,data/ptbxl/records100/00000/00003_lr,data/ptbxl/records500/00000/00003_hr
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,data/ptbxl/records100/00000/00004_lr,data/ptbxl/records500/00000/00004_hr
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,data/ptbxl/records100/00000/00005_lr,data/ptbxl/records500/00000/00005_hr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,"ventrikulÄre extrasystole(n) sinustachykardie linkstyp mÄssige amplitudenkriterien fÜr linkshypertrophie st & t abnorm, wahrscheinlich anterolaterale ischÄmie oder linksbelastung 4.46 unbestÄtigter bericht",...,True,NaN,", alles,",NaN,NaN,1ES,NaN,7,data/ptbxl/records100/21000/21833_lr,data/ptbxl/records500/21000/21833_hr
21834,20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm inferiorer infarkt wahrscheinlich alt 4.46 unbestÄtigter bericht,...,True,NaN,NaN,NaN,NaN,NaN,NaN,4,data/ptbxl/records100/21000/21834_lr,data/ptbxl/records500/21000/21834_hr
21835,19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,"sinusrhythmus lagetyp normal t abnorm in anterioren ableitungen 4.46 unbestÄtigter bericht Edit: INJAS 50, (ISCAS)",...,True,NaN,", I-AVR,",NaN,NaN,NaN,NaN,2,data/ptbxl/records100/21000/21835_lr,data/ptbxl/records500/21000/21835_hr


In [78]:
df_metadata.columns

Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')

### ECG Data

tbd description

In [79]:
def load_single_ecg(filepath:str):
    '''
    Load a single ECG file.
    :param filepath: Path to the ECG file.
    :return: The ECG signal as a numpy array (e.g. (1000, 12) for 1000 samples and 12 leads).
    '''
    signal, _ = wfdb.rdsamp(filepath)
    return signal

In [80]:
def load_ecg_data(df:pd.DataFrame, sampling_rate:int, max_workers:int=6) -> np.ndarray:
    '''
    Load raw ECG data from the PTB-XL database parallel.

    :param df: DataFrame containing metadata of ECG files
    :param sampling_rate: Sampling rate of the ECG data (100Hz or 500Hz)
    :param max_workers: Number of parallel workers to use for loading ECG files
    :return: Numpy array of ECG signals
    '''
    # lr = Low Resolution, 100Hz, hr = High Resolution, 500Hz
    filenames_col = 'filename_lr' if sampling_rate == 100 else 'filename_hr'
    filepaths = df[filenames_col].tolist()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        signals = list(tqdm(
            executor.map(load_single_ecg, filepaths),
            total=len(filepaths),
            desc=f"Loading {sampling_rate}Hz ECG data",
            unit="files"
        ))

    return np.array(signals)

In [81]:
# The X for machine learning
data_100hz = load_ecg_data(df_metadata, 100)

Loading 100Hz ECG data: 100%|██████████| 21799/21799 [01:09<00:00, 313.96files/s]


In [66]:
# The X for machine learning
# data_500hz = load_ecg_data(df_metadata, 500)

### SCP Codes

tbd description

In [82]:
df_scp_statements = pd.read_csv(path+'scp_statements.csv', index_col=0)
df_scp_statements

,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnormalities,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
DIG,digitalis-effect,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,suggests digitalis-effect,205.0,NaN,NaN,NaN
LNGQT,long QT-interval,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,long QT-interval,148.0,NaN,NaN,NaN
NORM,normal ECG,1.0,NaN,NaN,NORM,NORM,Normal/abnormal,normal ECG,1.0,NaN,NaN,F-000B7
...,...,...,...,...,...,...,...,...,...,...,...,...
BIGU,"bigeminal pattern (unknown origin, SV or Ventricular)",NaN,NaN,1.0,NaN,NaN,Statements related to ectopic rhythm abnormalities,"bigeminal pattern (unknown origin, SV or Ventricular)",NaN,NaN,NaN,NaN
AFLT,atrial flutter,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnormalities),atrial flutter,51.0,MDC_ECG_RHY_ATR_FLUT,NaN,NaN
SVTAC,supraventricular tachycardia,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnormalities),supraventricular tachycardia,55.0,MDC_ECG_RHY_SV_TACHY,NaN,D3-31290
PSVT,paroxysmal supraventricular tachycardia,NaN,NaN,1.0,NaN,NaN,Statements related to impulse formation (abnormalities),paroxysmal supraventricular tachycardia,NaN,MDC_ECG_RHY_SV_TACHY_PAROX,NaN,NaN


In [83]:
# filter statements to only include diagnostic statements
# a SCP code can be a diagnostic, form and/or a rythm statement
df_scp_statements = df_scp_statements[df_scp_statements['diagnostic'] == 1]
display(df_scp_statements)

,description,diagnostic,form,rhythm,diagnostic_class,diagnostic_subclass,Statement Category,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
NDT,non-diagnostic T abnormalities,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
NST_,non-specific ST changes,1.0,1.0,NaN,STTC,NST_,Basic roots for coding ST-T changes and abnormalities,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
DIG,digitalis-effect,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,suggests digitalis-effect,205.0,NaN,NaN,NaN
LNGQT,long QT-interval,1.0,1.0,NaN,STTC,STTC,other ST-T descriptive statements,long QT-interval,148.0,NaN,NaN,NaN
NORM,normal ECG,1.0,NaN,NaN,NORM,NORM,Normal/abnormal,normal ECG,1.0,NaN,NaN,F-000B7
IMI,inferior myocardial infarction,1.0,NaN,NaN,MI,IMI,Myocardial Infarction,inferior myocardial infarction,161.0,NaN,NaN,NaN
ASMI,anteroseptal myocardial infarction,1.0,NaN,NaN,MI,AMI,Myocardial Infarction,anteroseptal myocardial infarction,165.0,NaN,NaN,NaN
LVH,left ventricular hypertrophy,1.0,NaN,NaN,HYP,LVH,Ventricular Hypertrophy,left ventricular hypertrophy,142.0,NaN,C71076,NaN
LAFB,left anterior fascicular block,1.0,NaN,NaN,CD,LAFB/LPFB,Intraventricular and intra-atrial Conduction disturbances,left anterior fascicular block,101.0,MDC_ECG_BEAT_BLK_ANT_L_HEMI,C62267,D3-33140
ISC_,non-specific ischemic,1.0,NaN,NaN,STTC,ISC_,Basic roots for coding ST-T changes and abnormalities,ischemic ST-T changes,226.0,NaN,NaN,NaN


In [84]:
# e.g. for dict_scp_codes: {'NORM': 100, 'MI': 80}

def aggregate_diagnostic(dict_scp_codes: dict) -> list:
    '''
    Lookup every SCP Code in the dataframe and add scp code in a unique list.
    '''
    list_scp_codes = []

    for key in dict_scp_codes.keys():
        if key in df_scp_statements.index:
            list_scp_codes.append(df_scp_statements.loc[key]['diagnostic_class'])
    
    # remove duplicates by converting to a set and back to a list
    list_scp_codes = list(set(list_scp_codes))

    return list_scp_codes

In [85]:
# add column diagnostic superclass
tqdm.pandas(desc="Processing diagnostic superclass")
df_metadata['diagnostic_superclass'] = df_metadata['scp_codes'].progress_apply(aggregate_diagnostic)

Processing diagnostic superclass: 100%|██████████| 21799/21799 [00:02<00:00, 10062.85it/s]


In [90]:
# first step for a simple neuronal network
df_metadata['mi_label'] = df_metadata['diagnostic_superclass'].apply(lambda x: 1 if 'MI' in x else 0)

In [91]:
df_metadata

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,diagnostic_superclass,mi_label
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,", I-V1,",NaN,NaN,NaN,NaN,3,data/ptbxl/records100/00000/00001_lr,data/ptbxl/records500/00000/00001_hr,[NORM],0
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,NaN,NaN,NaN,NaN,NaN,2,data/ptbxl/records100/00000/00002_lr,data/ptbxl/records500/00000/00002_hr,[NORM],0
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,NaN,5,data/ptbxl/records100/00000/00003_lr,data/ptbxl/records500/00000/00003_hr,[NORM],0
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,NaN,3,data/ptbxl/records100/00000/00004_lr,data/ptbxl/records500/00000/00004_hr,[NORM],0
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,NaN,4,data/ptbxl/records100/00000/00005_lr,data/ptbxl/records500/00000/00005_hr,[NORM],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,"ventrikulÄre extrasystole(n) sinustachykardie linkstyp mÄssige amplitudenkriterien fÜr linkshypertrophie st & t abnorm, wahrscheinlich anterolaterale ischÄmie oder linksbelastung 4.46 unbestÄtigter bericht",...,", alles,",NaN,NaN,1ES,NaN,7,data/ptbxl/records100/21000/21833_lr,data/ptbxl/records500/21000/21833_hr,[STTC],0
21834,20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm inferiorer infarkt wahrscheinlich alt 4.46 unbestÄtigter bericht,...,NaN,NaN,NaN,NaN,NaN,4,data/ptbxl/records100/21000/21834_lr,data/ptbxl/records500/21000/21834_hr,[NORM],0
21835,19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,"sinusrhythmus lagetyp normal t abnorm in anterioren ableitungen 4.46 unbestÄtigter bericht Edit: INJAS 50, (ISCAS)",...,", I-AVR,",NaN,NaN,NaN,NaN,2,data/ptbxl/records100/21000/21835_lr,data/ptbxl/records500/21000/21835_hr,[STTC],0


## Train, Validation, Test Split

tbd description

In [87]:
X = data_100hz # OR data_500hz !
Y = df_metadata

In [89]:
print("Distribution diagnostic_superclass:\n", df_metadata['diagnostic_superclass'].value_counts(normalize=True))

Distribution diagnostic_superclass:
 diagnostic_superclass
[NORM]                 0.416028
[MI]                   0.116152
[STTC]                 0.110097
[CD]                   0.078352
[MI, CD]               0.059498
[STTC, HYP]            0.035827
[MI, STTC]             0.027478
[HYP]                  0.024542
[CD, STTC]             0.021606
[]                     0.018854
[CD, NORM]             0.018671
[MI, STTC, HYP]        0.016560
[CD, HYP]              0.013762
[MI, CD, STTC]         0.010230
[CD, STTC, HYP]        0.009679
[MI, HYP]              0.008395
[MI, CD, STTC, HYP]    0.007156
[MI, CD, HYP]          0.005367
[STTC, NORM]           0.001284
[CD, STTC, NORM]       0.000229
[CD, NORM, HYP]        0.000092
[NORM, HYP]            0.000092
[MI, CD, NORM, HYP]    0.000046
Name: proportion, dtype: float64


### Test Split

In [25]:
# 20% data for testing, 80% for training and validation
df_train_valid, df_test = train_test_split(
    df_metadata,
    test_size=0.2,
    stratify=df_metadata['mi_label'],
    random_state=42
)

In [26]:
print("Distribution of MI in df_train_valid:\n", df_train_valid['mi_label'].value_counts(normalize=True))
print('---')
print("Distribution of MI in df_test:\n", df_test['mi_label'].value_counts(normalize=True))

Distribution of MI in df_train_valid:
 mi_label
0    0.749126
1    0.250874
Name: proportion, dtype: float64
---
Distribution of MI in df_test:
 mi_label
0    0.749083
1    0.250917
Name: proportion, dtype: float64


### Train and Validation Split

In [27]:
# 20% for test, 10% für validation, 70% for training
df_train, df_val = train_test_split(
    df_train_valid,
    test_size=0.125, # 10% of the original data
    stratify=df_train_valid['mi_label'],
    random_state=42
)

In [28]:
print("Distribution of MI in df_train:\n", df_train['mi_label'].value_counts(normalize=True))
print('---')
print("Distribution of MI in df_valid:\n", df_val['mi_label'].value_counts(normalize=True))

Distribution of MI in df_train:
 mi_label
0    0.749132
1    0.250868
Name: proportion, dtype: float64
---
Distribution of MI in df_valid:
 mi_label
0    0.749083
1    0.250917
Name: proportion, dtype: float64


In [29]:
print(f"Size of Train-Set: {len(df_test)}")
print(f"Size of Validation-Set: {len(df_val)}")
print(f"Size of Test-Set: {len(df_train)}")

Size of Train-Set: 4360
Size of Validation-Set: 2180
Size of Test-Set: 15259


## Z-Score-Nomalization (μ, σ)

In [30]:
# turn array into one long 1D array
flattened_train = data_100hz.flatten() 
# flattened_train = data_500hz.flatten() 

In [31]:
# check data type
flattened_train = flattened_train.astype(np.float32)

In [32]:
# calculate meana and standard deviation
train_mean = np.mean(flattened_train)
train_std = np.std(flattened_train)

In [33]:
# Prevend division by zero
# if the standard deviation is very small, set it to a small positive value
if train_std < 1e-6:
    train_std = 1e-6

In [34]:
print(f"Calculated mean (μ) on training data: {train_mean:.6f}")
print(f"Calculated standard deviation (σ) on training data: {train_std:.6f}")

Calculated mean (μ) on training data: -0.000781
Calculated standard deviation (σ) on training data: 0.235392


In [35]:
# save values for normalization on the test and validation set
global_train_mean = train_mean
global_train_std = train_std

## Definition of PyTorch Dataset Class

tbd description

In [36]:
class ECGDataset(Dataset):
    '''
    Custom PyTorch Dataset class for loading and preprocessing ECG data.
    '''
    
    def __init__(self, df:pd.DataFrame, mean:float, std:float):
        '''
        Constructor for ECGDataset.

        :param df: DataFrame containing ECG metadata and paths to the ECG files.
        :param mean: Mean value (μ) from training data for normalization.
        :param std: Standard deviation (σ) from training data for normalization.
        '''
        self.df = df
        self.mean = mean
        self.std = std
        self.std_stable = self.std + 1e-8  # Prevent division by zero
    

    def __len__(self):
        '''
        Returns the number of samples in the dataset.

        :return: Number of samples.
        '''
        return len(self.df)


    def __getitem__(self, idx):
        '''
        Loads and preprocesses a single ESG sample based on its index.
        Called by the PyTorch DataLoader when it needs a new sample.

        :param idx: Index of the sample to load.
        :return: A tuple containing the ECG signal as a PyTorch tensor and the corresponding label.
        '''

        # 1. Get information from first sample
        row = self.df.iloc[idx]
        filepath = row['filename_lr']  # or 'filename_hr' for 500Hz
        label = row['mi_label']

        # 2. Load ECG data
        signal, _ = wfdb.rdsamp(filepath)

        # 3. Flatten the signal to a 1D array
        signal = signal.flatten()

        # 4. Normalize the signal (Z-Score-Normalization: (X - μ) / σ)
        signal = (signal - self.mean) / self.std_stable

        # 5. Converto to PyTorch tensor
        signal_tensor = torch.tensor(signal, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)

        return signal_tensor, label_tensor

In [37]:
train_ECGDataset = ECGDataset(df=df_train, mean=global_train_mean, std=global_train_std)
val_ECGDataset = ECGDataset(df=df_val, mean=global_train_mean, std=global_train_std)
test_ECGDataset = ECGDataset(df=df_test, mean=global_train_mean, std=global_train_std)

In [38]:
print(f"Size of training dataset: {len(train_ECGDataset)} Samples")
print(f"Size of validation dataset: {len(val_ECGDataset)} Samples")
print(f"Size of testing dataset: {len(test_ECGDataset)} Samples")

Size of training dataset: 15259 Samples
Size of validation dataset: 2180 Samples
Size of testing dataset: 4360 Samples


In [39]:
# Check if loading a single sample works and is correct
try:
    sample_signal, sample_label = train_ECGDataset[0]
    print(f"First sample:")
    print(f"Signal Shape: {sample_signal.shape}")
    print(f"Signal Mean: {sample_signal.mean().item():.4f}")
    print(f"Signal Std: {sample_signal.std().item():.4f}")
    print(f"Label: {sample_label.item()}")
except Exception as e:
    print(f"Error: {e}")

First sample:
Signal Shape: torch.Size([12000])
Signal Mean: 0.0014
Signal Std: 0.6086
Label: 0.0


## Create PyTorch DataLoaders

tbd description

In [40]:
# set the size of how many samples in the neuronal network are processed at once bevore the weights are updated
# 64 or 128 are common start values
# the bigger the batch size,
BATCH_SIZE = 64

num_workers = 0 # because of some windows issues with multiprocessing

In [41]:
train_loader = DataLoader(
    dataset=train_ECGDataset,
    batch_size=BATCH_SIZE,
    shuffle=True, # Shuffle the trainings data to improve generalization
    num_workers=num_workers
)

In [42]:
val_loader = DataLoader(
    dataset=val_ECGDataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # No Suffle, to ensure the same order of evaluation
    num_workers=num_workers
)

In [43]:
test_loader = DataLoader(
    dataset=test_ECGDataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # No Suffle, to ensure the same order of evaluation
    num_workers=num_workers
)

In [44]:
# check if the DataLoader works and returns the expected shapes and values
try:
    # 'iter(train_loader)' to iterate over the dataloader like a list
    # 'next(...)' gets the next batch, a pair of signal tensor and label tensor
    first_batch_signals, first_batch_labels = next(iter(train_loader))

    print(f"Shape of signal batch (should be (Batch_Size, flattened_ECG_length)): {first_batch_signals.shape}")
    print(f"Shape of label batch (should be (Batch_Size,) or (Batch_Size, 1)): {first_batch_labels.shape}")

    print(f"Mean of the first batch (signal): {first_batch_signals.mean().item():.4f}")
    print(f"Standard deviation of the first batch (signal): {first_batch_signals.std().item():.4f}")
    
    print(f"First 5 labels of the batch: {first_batch_labels[:5].tolist()}")

except Exception as e:
    print(f"Error while testing the DataLoader: {e}")

Shape of signal batch (should be (Batch_Size, flattened_ECG_length)): torch.Size([64, 12000])
Shape of label batch (should be (Batch_Size,) or (Batch_Size, 1)): torch.Size([64])
Mean of the first batch (signal): 0.0005
Standard deviation of the first batch (signal): 0.9801
First 5 labels of the batch: [0.0, 1.0, 0.0, 1.0, 0.0]


## Simple Neuronal Network

### Architecture

In [ ]:
# set input features to the length of the flattened ECG signal
INPUT_SIZE = 12000 # Dies ist die Länge Ihres geflatteten EKG-Signals

In [ ]:
# choose hidden layers size
HIDDEN_SIZE_1 = 256
HIDDEN_SIZE_2 = 128

In [ ]:
# set output size to 1 because of binary classification
OUTPUT_SIZE = 1

In [ ]:
class SimpleNN(nn.Module):
    '''
    A simple feedforard neuronal network for binary ECG classification
    '''
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleNN, self).__init__()

        # --- Layers ---

        # 1. Fully Connected Layer 1: Connection between ECG features as input and the first hidden layer
        self.fc1 = nn.Linear(input_size, hidden_size1)

        # 2. Activation Function 1: ReLU (Rectified Linear Unit): Add non-linearity to the model to learn complex patterns
        # plot: ___/
        self.relu1 = nn.ReLU()

        # 3. Fully Connected Layer 3: Connection between the first and second hidden layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)

        # 4. Activation Function 2: ReLU
        self.relu2 = nn.ReLU()

        # 5. Output Layer: Connection between the second hidden layer and the output neuron
        self.fc3 = nn.Linear(hidden_size2, output_size)


    def forward(self, x):
        '''
        Forward pass of the network which describes how the input data
        flows through the definied layers.

        :param x: The input tensor (a batch of ECG signals)
        :return: The output of the network (the "logits" or raw scores)
        '''
        # Feed the data through the first fully connected layer
        x = self.fc1(x)
        x = self.relu1(x)

        # Feed the data through the second fully connected layer
        x = self.fc2(x)
        x = self.relu2(x)
        
        # Feed the data trough the output layer
        x = self.fc3(x)
        
        return x

### Initialization 

In [ ]:
# Check if GPU (CUDA) is available and use it
# If not, use CPU -> GPU is much faster for training neural networks
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

In [ ]:
# Create instance of the neural network
# and move it to the device (GPU or CPU)
model = SimpleNN(
    input_size=INPUT_SIZE,
    hidden_size1=HIDDEN_SIZE_1,
    hidden_size2=HIDDEN_SIZE_2,
    output_size=OUTPUT_SIZE
).to(device)

print(model)

In [ ]:
# Check how many trainable parameters the model has
# Metrics for model complexity

# parameter.numel() returns the number of elements in the parameter tensor
# model.parameters() returns the weights and biases of each layer as a 'list'
# parameter.requires_grad is True for trainable parameters -> it shows the direction and how much the weights should be updated to perform better
total_params = sum(parameter.numel() for parameter in model.parameters() if parameter.requires_grad)
print(f'Total number of trainable parameters in the model: {total_params}')

### Loss Function and Optimizer

In [ ]:
# for binary classification with logits is 'BCEWithLogitsLoss' commonly used
# Combinaton of Sigmoid function and Binary Cross Entropy Loss
# Output are probabilities
criterion_lossf = nn.BCEWithLogitsLoss()

In [ ]:
# Optimizer 'Adam' with parameters:
# model.parameters(): Which parameter to optimze (all trainable parameters of the network)
# lr (learning rate): The learning rate is a hyperparameter that controls how large the steps are that the optimizer takes to adjust the weights.
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Training-Loop

1. Forward Pass: Pass input data (ECG-Batch) trough neuronal network for prediction
2. Loss Calculation: Compare the prediction with the true label
3. Backpropagation: Calculate gradients and update weights with pytorch
4. Optimier: Use gradients to update the weights and minimize the loss
5. Get metrics from training

In [ ]:
#  Set the number of epochs for training
NUM_EPOCHS = 10 

In [ ]:
# check model
model.to(device)

In [ ]:
# save loss and accuracy for each epoch for analysis and plotting
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
# main training loop

for epoch in range(NUM_EPOCHS):

    # Set the model to training mode
    model.train()

    current_train_loss = 0.0
    correct_train_predictions = 0
    total_train_samples = 0

    # move trough all batches in training dataloader
    for batch_idx, (inputs, labels) in enumerate(train_loader):

        # --- TRAINING ---
        # move data and labels to device (GPU or CPU)
        inouts = inputs.to(device)
        labels = labels.to(device)

        # set gradient to zero (pytorch accumulates gradients by default)
        optimizer.zero_grad()

        # 1. Forward Pass
        outputs = model(inputs)

        # 2. Loss Calculation
        # squeeze() removes dimensions of size 1 from the output tensor
        # labels should be a 1D tensor for BCEWithLogitsLoss
        loss = criterion_lossf(outputs.squeeze(), labels)

        # 3. Backpropagation
        loss.backward()

        # 4. Optimizer
        optimizer.step()

        # 5. Get metrics from training
        # loss.item() return the value of the loss
        # inputs.size(0) is the actual batch size, which can be smaller than BATCH_SIZE in the last batch
        current_train_loss += loss.item() * inputs.size(0)

        # For Accuracy Calculation
        # BCEWithLogitsLoss outputs logits -> apply sigmoid to get probabilities
        # set treshhold to 0.5 for binary classification
        predicted_probs = torch.sigmoid(outputs)
        predicted_classes = (predicted_probs > 0.5).float()
        correct_train_predictions += (predicted_classes.squeeze() == labels).sum().item()
        total_train_samples += labels.size(0)
    
    # Calulcate average trainings loss and accuracy for the epoch
    avg_train_loss = current_train_loss / total_train_samples
    train_accuracy = (correct_train_predictions / total_train_samples) * 100
    
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)



    # --- VALIDATION ---
    # set model to evaluation mode
    model.eval()

    current_val_loss = 0.0
    correct_val_predictions = 0
    total_val_samples = 0

    # Deactivate gradient calculation to save memory and speed up validation
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward Pass (no backward(), no optimizer.step()!)
            outputs = model(inputs)

            # calculate loss
            loss = criterion_lossf(outputs.squeeze(), labels)
            current_val_loss += loss.item() * inputs.size(0)

            # Collect metrics for validation
            predicted_probs = torch.sigmoid(outputs)
            predicted_classes = (predicted_probs > 0.5).float()

            correct_val_predictions += (predicted_classes.squeeze() == labels).sum().item()
            total_val_samples += labels.size(0)
        
    # Calculate average validation loss and accuaracy for the epoch
    avg_val_loss = current_val_loss / total_val_samples
    val_accuracy = (correct_val_predictions / total_val_samples) * 100
    
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)



    # --- PRINT RESULTS ---
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")


print("\nTraining complete!")

# --- Optional: Modell speichern ---
# Wenn Sie das trainierte Modell später wiederverwenden möchten.
torch.save(model.state_dict(), 'simple_nn_ekg_model.pth')
print("Saved the trained model to 'simple_nn_ekg_model.pth'")


## xresnet1d101

### Create DataLoaders with fastai

In [ ]:
# fastai verwendet eine eigene DataLoaders-Klasse, die Ihre PyTorch DataLoaders verpackt.
# Dies ist der erste Punkt, an dem wir von reinem PyTorch zu fastai wechseln.
dls = DataLoaders(train_loader, val_loader)

## Test pretrained resnet model

In [45]:
# Neue Zelle nach "## Test pretrained resnet model"

import torch
from tsai.all import *
import numpy as np
from pathlib import Path

# Pfad zum vortrainierten Modell
model_path = Path(r"c:\Users\AliceKitchkin\Repos\ecg_ptbxl_benchmarking\output\exp0\models\fastai_xresnet1d101\models\fastai_xresnet1d101.pth")

# Überprüfen ob Modell existiert
if model_path.exists():
    print(f"✅ Modell gefunden: {model_path}")
    # Modell laden
    pretrained_model = torch.load(model_path, map_location='cpu')
    print("✅ Modell erfolgreich geladen!")
else:
    print(f"❌ Modell nicht gefunden: {model_path}")
    print("Überprüfen Sie den Pfad oder laden Sie das Modell herunter.")

✅ Modell gefunden: c:\Users\AliceKitchkin\Repos\ecg_ptbxl_benchmarking\output\exp0\models\fastai_xresnet1d101\models\fastai_xresnet1d101.pth
✅ Modell erfolgreich geladen!


In [46]:
# Neue Zelle: Daten für XResNet1d vorbereiten

# Das vortrainierte Modell erwartet spezifische Eingabeformate
# Normalerweise: (batch_size, channels, sequence_length)

def prepare_data_for_xresnet(df_metadata, sampling_rate=100):
    """
    Bereitet ECG-Daten für XResNet1d vor
    """
    # Laden der ECG-Signale (verwenden Sie Ihre bestehende Funktion)
    ecg_data = load_ecg_data(df_metadata, sampling_rate)
    
    # XResNet1d erwartet (N, C, L) Format
    # N = Anzahl Samples, C = Channels (ECG Leads), L = Sequence Length
    if len(ecg_data.shape) == 3:
        # Bereits im richtigen Format (samples, time, leads)
        ecg_data = ecg_data.transpose(0, 2, 1)  # (samples, leads, time)
    else:
        # Falls flattened, zurück zu 3D
        ecg_data = ecg_data.reshape(len(df_metadata), 1000, 12)
        ecg_data = ecg_data.transpose(0, 2, 1)
    
    return ecg_data

# Vorbereitung der Daten
print("Bereite Daten für XResNet1d vor...")
X_xresnet = prepare_data_for_xresnet(df_train.head(100))  # Test mit 100 Samples
print(f"Shape für XResNet1d: {X_xresnet.shape}")  # Sollte (100, 12, 1000) sein

Bereite Daten für XResNet1d vor...


Loading 100Hz ECG data: 100%|██████████| 100/100 [00:00<00:00, 261.46files/s]

Shape für XResNet1d: (100, 12, 1000)


In [51]:
# Neue Zelle: XResNet1d Learner erstellen

from tsai.models.XResNet1d import xresnet1d101
from fastai.learner import Learner

# Erstellen Sie einen neuen XResNet1d101
model_xresnet = xresnet1d101(
    c_in=12,        # 12 ECG Leads
    c_out=1,        # Anzahl Klassen (MI/No MI)
    #seq_len=1000    # Sequence Length (1000 für 100Hz, 5000 für 500Hz)
)

# Laden der vortrainierten Gewichte
if model_path.exists():
    # State Dict laden und anpassen
    pretrained_state = torch.load(model_path, map_location='cpu')
    
    # Falls das Modell ein FastAI Learner war
    if hasattr(pretrained_state, 'model'):
        model_state = pretrained_state.model.state_dict()
    else:
        model_state = pretrained_state
    
    # Gewichte in neues Modell laden
    try:
        model_xresnet.load_state_dict(model_state, strict=False)
        print("✅ Vortrainierte Gewichte erfolgreich geladen!")
    except Exception as e:
        print(f"⚠️ Warnung beim Laden der Gewichte: {e}")
        print("Versuche kompatible Gewichte zu laden...")

print(f"Modell-Architektur: {model_xresnet}")

✅ Vortrainierte Gewichte erfolgreich geladen!
Modell-Architektur: XResNet(
  (0): ConvLayer(
    (0): Conv1d(12, 32, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): ConvLayer(
    (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): ConvLayer(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convpath): Sequential(
        (0): ConvLayer(
          (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1

In [54]:
# Neue Zelle: Vorhersagen mit vortrainiertem Modell

def predict_with_xresnet(model, ecg_data, device='cpu'):
    """
    Macht Vorhersagen mit dem XResNet1d Modell
    """
    model.eval()
    model.to(device)
    
    with torch.no_grad():
        if not isinstance(ecg_data, torch.Tensor):
            ecg_data = torch.tensor(ecg_data, dtype=torch.float32)
        
        ecg_data = ecg_data.to(device)
        
        # Vorhersage (Logits)
        outputs = model(ecg_data)
        
        # Sigmoid für Wahrscheinlichkeiten (binäre Klassifikation)
        probabilities = torch.sigmoid(outputs)
        
        # Klassen-Vorhersagen (Threshold 0.5)
        predictions = (probabilities > 0.5).float()
        
    return predictions.cpu().numpy(), probabilities.cpu().numpy()

# Test-Vorhersagen
test_data = X_xresnet[:10]  # Erste 10 Samples
predictions, probabilities = predict_with_xresnet(model_xresnet, test_data)

print(f"Vorhersagen für erste 10 Samples: {predictions}")
print(f"Wahrscheinlichkeiten:\n{probabilities}")

Vorhersagen für erste 10 Samples: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Wahrscheinlichkeiten:
[[0.41724533]
 [0.3611281 ]
 [0.21777542]
 [0.41217253]
 [0.3949846 ]
 [0.431964  ]
 [0.4128642 ]
 [0.40053928]
 [0.41038045]
 [0.40012234]]


In [55]:
# Neue Zelle: Modell auf eigenen Daten evaluieren

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Vorbereitung der Test-Daten
X_test_xresnet = prepare_data_for_xresnet(df_test)
y_test = df_test['mi_label'].values

# Vorhersagen auf Test-Set
test_predictions, test_probabilities = predict_with_xresnet(model_xresnet, X_test_xresnet)

# Evaluation
accuracy = accuracy_score(y_test, test_predictions)
print(f"Accuracy auf Test-Set: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, test_predictions, target_names=['No MI', 'MI']))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, test_predictions))

Loading 100Hz ECG data: 100%|██████████| 4360/4360 [00:14<00:00, 311.05files/s]


Accuracy auf Test-Set: 0.7491

Classification Report:
              precision    recall  f1-score   support

       No MI       0.75      1.00      0.86      3266
          MI       0.00      0.00      0.00      1094

    accuracy                           0.75      4360
   macro avg       0.37      0.50      0.43      4360
weighted avg       0.56      0.75      0.64      4360


Confusion Matrix:
[[3266    0]
 [1094    0]]
